# Constructing ghosts

The purpose of this notebook is to provide a general algorithm for constructing ghosts assuming the Stark conjectures and the twisted convolution identity. We focus (for now) on the case of rank 1 ghosts. We also leave necromancy for future work. 

This also serves as a test of the software package Zauner.gp. 

To load PARI/GP togeher with the Jupyter kernel that powers this notebook, use Conda and type

`conda install pari_jupyter -c conda-forge`

As of this writing, this should install PARI/GP version 2.13.3 into your current virtual environment together with the Jupyter kernel. 

In [1]:
1+1

2

In [1]:
read("../src/Zauner.gp")

Zauner.gp  v0.0.1


What defines a ghost to begin with? 
Conjecturally, a ghost can be specified by:
- An integer $d > 3$. This is the dimension of the underlying complex vector space, $\mathbb{C}^d$. 
- A quadratic form $Q$ of discriminant $\Delta := f^2 \Delta_0$ where $\Delta \big\vert (d+1)(d-3)$ and $\Delta_0$ is a fundamental discriminant. 

_Remark:_ While the ghost formula depends explicitly on $Q$, different $Q$ in the same class give ghosts on the same extended Clifford orbit. 
Therefore, we can choose $Q$ to be a reduced form wlog. 
This choice is motivated by the fact that non-reduced forms require greater computational resources in general.

_Remark:_ To define a SIC, we would also (conjecturally) need to specify a sign-switching Galois automorphism $g(\Delta_0) = -\Delta_0$. 
This could be specified by an explicit choice of minimal polynomial for the field containing the ghost as well as an interval containing a root. 

Because we need quadratic forms of a given discriminant, it is germane to list all forms of a fixed discriminant, so let's start with that. 

In [2]:
d = 4;
[Delta0,f] = D0(d);
allf = divisors(f);

fq = allf[1]; \\ a divisor of f.
[qb,p,c] = ghostbasis(d,fq^2*Delta0);

Q = qb[1]^1;

L = stabilizer(Q);
a = stabilizersl2order(L,d);
A = L^a;

{
    my(a,b,c);
    [a,b,c] = Vec(Q);
    w = quadgen(b^2-4*a*c,'w); \\ quadratic generator for disc_Q
    beta = (-b+2*w-(b^2%4))/(2*a) \\ a positive root of Q
}


print("\nd = ", d, ", Δ = ", Delta0,"\nf = ", allf, "\nConductor fq = ", fq, "\nQ basis = ", [qb,p,c], "\nQ = ", Q, "\nA = psl2word(", psl2word(A), ")\n");



d = 4, Δ = 5
f = [1]
Conductor fq = 1
Q basis = [[Qfb(1, 1, -1, 0.E-38)], [1], 1]
Q = Qfb(1, 1, -1, 0.E-38)
A = psl2word([1, 3, 3, 2])



In [13]:
allsics(d) = {
    my( Delta0, f, qb, p, c, r, Q = List());

    [Delta0,f] = D0(d);
    f = divisors(f);
    
    for( j=1, #f,
        [qb, p, c] = ghostbasis(d,f[j]^2*Delta0);
        for( k=0, c-1,  
            r = radix(k,p);
            listput(Q,prod( n=1, #r, qb[n]^(r[n]) ) );
        );
    );
    Vec(Q)
}

qdata(Q,d) = {
    my(L, A, a, b, c);

    L = stabilizer(Q);
    a = stabilizersl2order(L,d);
    A = L^a;

    [a,b,c] = Vec(Q);
    w = quadgen(b^2-4*a*c,'w); \\ quadratic generator for disc_Q
    beta = (-b+2*w-(b^2%4))/(2*a); \\ a positive root of Q
    [A,beta]
}

ghostunittest(dmin,dmax,eps=1E-16) = 
{
    my(t = 0, QQ, A, beta, allnu, G, d=dmin, projtest);
    
    while( (d <= dmax) && (t < eps), 
        QQ = allsics(d);
        print("Checking ",#QQ," cases for d = ",d,".");
        for( j=1, #QQ, 
            Q = QQ[1];
            [A,beta] = qdata(Q,d);
            allnu = matrix(d,d,p1,p2, nu(A,d,[p1-1,p2-1],beta,[0,0]) );
            G = sum(p1=0,d-1, sum(p2=0,d-1, allnu[p1+1,p2+1]*WH(p1,p2,d) ));
            projtest = G^2-d*G;
            t = sqrt(norml2(projtest))/d^2;
            print(t);
        );
        d = d+1;
    );
}

time = 1 ms

In [13]:
ghostunittest(4,10)

Checking 2 cases for d = 9.
1.2323313216367722520 E-20
1.2323313216367722520 E-20
Checking 1 cases for d = 10.
2.860066709939282734 E-19
Checking 3 cases for d = 11.
3.509488860386916639 E-20
3.509488860386916639 E-20
3.509488860386916639 E-20
time = 2min, 38,719 ms

In [6]:
\\d = 4;
QQ = allsics(d);
print("Checking ",#QQ," cases for d = ",d,".");
for( j=1, #QQ, {
    Q = QQ[1];
    [A,beta] = qdata(Q,d);
    allnu = matrix(d,d,p1,p2, nu(A,d,[p1-1,p2-1],beta,[0,0]) );
    G = sum(p1=0,d-1, sum(p2=0,d-1, allnu[p1+1,p2+1]*WH(p1,p2,d) ));
    projtest = G^2-d*G;
    print(sqrt(norml2(projtest))/d^2);
});
d = d+1

Checking 1 cases for d = 6.
3.363991524326761769 E-25


7

time = 5,953 ms

In [51]:
allnu = matrix(d,d,p1,p2, nu(A,d,[p1-1,p2-1],beta,[0,0]) );
G = sum(p1=0,d-1, sum(p2=0,d-1, allnu[p1+1,p2+1]*WH(p1,p2,d) ));
projtest = G^2-d*G;
norml2(projtest)


time = 43,473 ms

1.2330802691730499100455052965036529482 E-24

In [106]:
vecmax(vector(100000,k,towerh(k+3)))

11

time = 1,537 ms

In [156]:
lindep([G[2,3],1,sqrt(3)])

[0, 1385331749802026, -799821658665135]~

(v)->1

In [111]:
v.rs

Qfb(1, 1, -1, 0.E-38)

In [109]:
v.rs = Qfb(1,1,-1)

(v)->Qfb(1,1,-1)

In [18]:
??quadclassunit

quadclassunit(D,{flag = 0},{tech = []}):

   Buchmann-McCurley's  sub-exponential algorithm for computing the class group
of a quadratic order of discriminant D.

   This function should be used instead of qfbclassno or quadregulator when D <
-10^{25}, D > 10^{10}, or when the structure is wanted. It is a special case of
bnfinit, which is slower, but more robust.

   The  result  is  a vector v whose components should be accessed using member
functions:

* v.no: the class number

*  v.cyc:  a  vector  giving  the  structure of the class group as a product of
cyclic groups;

* v.gen: a vector giving generators of those cyclic groups (as binary quadratic
/*-- (type RETURN to continue) --*/
forms).

*  v.reg:  the  regulator,   computed to an accuracy which is the maximum of an
internal accuracy determined by the program and the current default  (note that
once  the regulator is known to a small accuracy it is trivial to compute it to
very high accuracy, see the tutorial).

   The  flag  

In [30]:
sicdatatable(d) = 
{
    my(D,s,f,ff,Delta,v);
    [D,s] = coredisc( (d+1)*(d-3), 1);
    f = divisors(s);
    ff = apply(sqr,f);
    Delta = ff*D;
    v = vector( #f, k, quadclassunit(Delta[k]) );
    for(k = 1, #v,
        if(v[k][1]==1, 
            v[k][2] = [1];
            v[k][3] = [prinredqfb(Delta[k])];
        );
    );
    v;
    
    h = vector(#f, k, towerh(d, Delta[k]));
    print(v,"\n",h)
}

(d)->my(D,s,f,ff,Delta,v);[D,s]=coredisc((d+1)*(d-3),1);f=divisors(s);ff=apply(sqr,f);Delta=ff*D;v=vector(#f,k,quadclassunit(Delta[k]));for(k=1,#v,if(v[k][1]==1,v[k][2]=[1];v[k][3]=[prinredqfb(Delta[k])];););v;h=vector(#f,k,towerh(d,Delta[k]));print(v,"\n",h)

In [30]:
sicdatatable(8)

[[1, [1], [Qfb(1, 1, -1, 0.E-38)], 0.48121182505960344749775891342436842314], [1, [1], [Qfb(1, 5, -5, 0.E-38)], 1.9248473002384137899910356536974736925]]
[2, 1]


In [102]:
sicclass(d) = 
{
    my(D,s,f,ff,Delta,v,t,u,h,a,b);
    [D,s] = coredisc( (d+1)*(d-3), 1);
    f = divisors(s);
    ff = apply(sqr,f);
    Delta = ff*D;
    v = vector( #f, k, quadclassunit(Delta[k]) );
    for(k = 1, #v,
        if(v[k][1]==1, 
            v[k][2] = [1];
            v[k][3] = [prinredqfb(Delta[k])];
        );
    );
    t = sum( k=1, #f, v[k][1] ); \\ total number in dimension d
    u = vector(#f, k, quadunit(Delta[k]));
    a = trace(u)/2;
    b = round(apply(sqrt,apply(sqr,u-a)/D));
    for(k = 1, #u,
        if(norm(u[k]) == -1, u[k] = u[k]^2);
    );
    h = round(acosh((d-1)/2)/log(u[1]));
    [d,D,h,t,f,v,u,a,b]
}


(d)->my(D,s,f,ff,Delta,v,t,u,h,a,b);[D,s]=coredisc((d+1)*(d-3),1);f=divisors(s);ff=apply(sqr,f);Delta=ff*D;v=vector(#f,k,quadclassunit(Delta[k]));for(k=1,#v,if(v[k][1]==1,v[k][2]=[1];v[k][3]=[prinredqfb(Delta[k])];););t=sum(k=1,#f,v[k][1]);u=vector(#f,k,quadunit(Delta[k]));a=trace(u)/2;b=round(apply(sqrt,apply(sqr,u-a)/D));for(k=1,#u,if(norm(u[k])==-1,u[k]=u[k]^2););h=round(acosh((d-1)/2)/log(u[1]));[d,D,h,t,f,v,u,a,b]

In [107]:
v = sicclass(7)

[7, 8, 1, 2, [1, 2], [[1, [1], [Qfb(1, 2, -1, 0.E-38)], 0.88137358701954302523260932497979230903], [1, [1], [Qfb(1, 4, -4, 0.E-38)], 1.7627471740390860504652186499595846181]], [3 + 2*w, 3 + w], [1, 3], [1, 1]]

In [98]:
apply(sqrtint,[1, 1, 100, 100, 100, 100])

[1, 1, 10, 10, 10, 10]

In [89]:
apply(sqr,[5, 5, 49, 49, 49, 49])/Delta0

[25/21, 25/21, 343/3, 343/3, 343/3, 343/3]

In [99]:
?sqrt

sqrt(x): square root of x.



In [ ]:
allbvals(A,beta) = 
{
    my( W, B = A, bvals = List([]) );

    W = psl2word(A);
    for( j=2, #W,
        B = [0,1;-1,W[j-1]]*B;
        listput(bvals, (B[1,1]*beta+B[1,2])/(B[2,1]*beta+B[2,2]) );
    );
    return( Vec(bvals) )
}

(A,beta)->my(W,B=A,bvals=List([]));W=psl2word(A);for(j=2,#W,B=[0,1;-1,W[j-1]]*B;listput(bvals,(B[1,1]*beta+B[1,2])/(B[2,1]*beta+B[2,2])););return(Vec(bvals))

The first step is to compute $L = L(Q)$, a generator of the stability group of $Q$, i.e. the group of matrices $M \in \mathrm{SL}_2(\mathbb{Z})$ such that $MQM^T = Q$. If we can write an arbitrary $M$ in this group as $M =\pm L^k$ for $k\in \mathbb{Z}$, then $L$ is a valid generator and any choice (there are four) gives us a choice of $L(Q)$. (The freedom in choice is $\pm L,\pm L^{-1}$.)

## Scratch workspace

This adds the q-Pochhamer symbol and the variant q-Pochhammer symbol.

In [26]:
[1, 1; 2, 3] % 2


[1 1]

[0 1]


In [29]:
qpe(.1,.5+(1E-100)*I,14)

2.9508497187473712051146708591409529426 - 0.95878919424435291979654916312937274643*I

In [15]:
L = [1,2;3,4]





[1 2]

[3 4]


In [31]:
sfjc(.1,.5+(1E-50)*I,[0,-1;1,0])

-3254.1236255738056835903768879695342069 - 7.553243618576152959 E-32*I

In [33]:
ds(2,4,6)-sqrt(2)

0.E-38 + 0.E-54*I

In [20]:
if(a==0,"True!","false.")

"false."

In [17]:
default(realprecision)

462

In [16]:
lindep([ds( 1/2, 1, (5+sqrt(21))/2 ), sqrt(2)])

[-1, 1]~

In [24]:
ds(1,4,6)

In [23]:
\p120
{
my(t = (5+sqrt(21))/2);
a = ds(1/3,1,t)*ds(1+t/3,1,t)*ds((2+2*t)/3,1,t) - sqrt((1+sqrt(21))/4 - sqrt((3+sqrt(21))/2)/2);
\\algdep(a,8)
abs(a) < 10^(-120)
}

1

In [8]:
?algdep

algdep(z,k,{flag=0}): algebraic relations up to degree n of z, using 
lindep([1,z,...,z^(k-1)], flag).



In [113]:
ds(1/3,1,t)*ds(1+t/3,1,t)*ds((2+2*t)/3,1,t) - 1.0*sqrt((1+sqrt(21))/4 - sqrt((3+sqrt(21))/2)/2)

1.7272337110188889251 E-77 + 0.E-77*I

In [6]:
{localprec(200);

print();
}

231


Here is a rewrite that tries to improve the precision by working in log space and pushing the z point closer to the real axis.

In [ ]:
\\ Double sine function
ds(w,b1,b2) = 
{
    if( b1*b2 == 0 , 
        error("Domain error, b1*b2 == 0.")
    );
    if( bitand(imag(b1/b2) == 0, real(b1/b2) < 0) ,
        error("Domain error, imag(b1/b2)==0 and real(b1/b2) < 0.");
    );
    
    \\ Make the second one the longer (absolute) period and normalize it to 1.
    if( abs(b2) < abs(b1), 
        return( exp(dsShift(w/b1,b2/b1)) );
        ,
        return( exp(dsShift(w/b2,b1/b2)) );
    )    
}

\\ Shift real(z) until 0 < real(z) < real(w)+1 and abs(imag(z)) < abs(imag(w))
dsShift(z,w) =
{
    \\ First minimize the distance to the real axis
    if( imag(w) > 0,
        if(imag(z) >= imag(w),
            return( dsShift(z-w, w)-log(2*sin(Pi*(z-w))) ),
        imag(z) <= -imag(w),
            return( dsShift(z+w, w)+log(2*sin(Pi*z)) )
        );,
    imag(w) < 0,
        if(imag(z) <= imag(w),
            return( dsShift(z-w, w)-log(2*sin(Pi*(z-w))) ),
        imag(z) >= -imag(w),
            return( dsShift(z+w, w)+log(2*sin(Pi*z)) )
        );,
    );
    
    
    \\ Now shift real(z) into the domain for the integral formula.
    if(z == 0,
        return( 0 ),
    real(z) <= 0, 
        return( dsShift(z+1, w)+log(2*sin(Pi*z/w)) ),
    real(z) >= real(w)+1,
        return( dsShift(z-1, w)-log(2*sin(Pi*(z-1)/w)) ),
    1,
        return( dsInt(z, w) )
    );
}

\\ Integral formula for the double sine function
dsInt(z,w) =
{    
    my( a, b, c, d, EPS);

    EPS = 1E-3; \\ hard code the boundary for now. In principle, this choice shouldn't matter.
    
    \\ integrate the power series around zero, up to EPS
    a = dsPowerSeries(z,w,EPS);

    \\ numerically integrate from EPS to 1
    b = intnum(t=EPS, 1, \
            (expm1(-t*z)-expm1(-t*(w+1-z)))/(t*(expm1(-w*t))*(expm1(-t))) \
             - (w+1-2*z)/(w*t^2));

    \\ integrate 1 to oo
    c = intnum(t=1, [+oo, min(real(z),real(w+1-z))], \
            (exp(-t*z)-exp(-t*(w+1-z)))/(t*(1-exp(-w*t))*(1-exp(-t)))); 

    \\ boundary term
    d = -(w+1-2*z)/w;

    -(a+b+c+d)
}

\\ Integrate the Double sine power series in the main domain from zero to eps.
dsPowerSeries(z,w,eps) = 
{
    my( isp = default(seriesprecision), csp = default(seriesprecision), sa, sb, a = 1, b = 0);
                        
    \\ keep doubling seriesprecision until convergence
    while( abs(a-b) > 10^(-default(realprecision)),
        default(seriesprecision, csp);
        \\ integrate up to csp terms.
        sa = intformal(serchop(Ser(sinh(((w+1)/2-z)*t)/(2*t*sinh(w*t/2)*sinh(t/2))-(w+1-2*z)/(w*t^2),t)));
        \\ explicitly enforce odd parity
        sa = (sa - subst(sa,t,-t))/2;
        \\ for comparison, truncate to order csp-4.
        sb = Ser(sa,t,csp-4);
        a = subst(truncate(sa),t,eps);
        b = subst(truncate(sb),t,eps);
        csp = 2*csp;
    );
    \\restore original seriesprecision and return a
    default(seriesprecision, isp);
    a
}


\\ exponential function
e(z) = exp(2*Pi*I*z)


Some unit tests for the double sine function using the exact values computed in Shintani's original paper:
T. Shintani, "On a Kronecker limit formula for real quadratic fields", J. Fac. Sci. Univ.
Tokyo 24 (1977), 167–199.

Shin-ya Koyama and Nobushige Kurokawa, "Values of the double sine function", Journal of Number Theory, 123 1 (2007), 204-223.
http://www1.tmtv.ne.jp/~koyama/recentpapers/values.pdf

and from
Nobushige Kurokawa and Masato Wakayama, "Algebraicity and transcendency of basic special values of Shintani’s double sine functions", Proceedings of the Edinburgh Mathematical Society 49, (2006) 361–366.


In [48]:
a = ds(2-sqrt(2),1,sqrt(2)) + 2.0^(5/4) * cos(Pi/sqrt(2)) 

-2.2454038243245556026 E-76 + 0.E-76*I

In [57]:
\p200

   realprecision = 211 significant digits (200 digits displayed)


In [21]:
ds(1,4,6) == 1.0

1

In [58]:
ds(2-sqrt(2),1,sqrt(2)) + 2.0^(5/4) * cos(Pi/sqrt(2)) 

-2.2454038243245556026 E-76 + 0.E-76*I

The next block of GP code is for the Shintani-Fadeev cocycle and Shin.

In [ ]:
\\ Shintani-Fadeev modular cocycle
sfcocycle(M, z, tau) = 
{
    my( a, b, c, d, S = [0, -1; 1, 0], k, L, s, t, u, v);
    
    a = M[1,1]; b = M[1,2]; c = M[2,1]; d = M[2,2];
            
    if( M == [1, 0; 0, 1],
            return( 1 );
        ,
        M == [-1, 0; 0, -1],
            error("M cannot equal -I.");
        ,
        c < 0,
            return( sfcocycle( M^-1, z/(c*tau+d), (a*tau+b)/(c*tau+d) )^(-1) );
        ,
        M == S,
            return( e((tau-3+tau^-1)/24+(tau-z)*(1-z)/(4*tau))*(1-e(z/tau))*ds(z,tau,1) );
        ,
        1,
            k = if( c == 0, 0, ceil(a/c) );
            L = [0,1; -1, k]*M; \\ S^-1*T^-k*M
            s = L[1,1]; t = L[1,2]; u = L[2,1]; v = L[2,2];
            return( sfcocycle(S, z/(u*tau+v), (s*tau+t)/(u*tau+v) )*sfcocycle(L, z, tau) );
    )
}
